In [2]:
from digicampipe.scripts.effective_area import get_fitted
from digicampipe.io.dl1 import read_hdf5, read_event_histogram 
from digicampipe.visualization.machine_learning import LABELS, plot_3d_histo, get_rate_gamma, get_rate_proton, compute_effective_area, crab_spectrum, proton_spectrum
from digicampipe.pointing.disp import angular_distance
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib
import numpy as np
import os
import pandas as pd
figsize=(12, 10)
import imp
imp.reload(matplotlib)
imp.reload(plt)
%matplotlib qt

In [55]:
def plot_trigger_rate(counts_events, counts_trigger, bins_energy, bins_impact, kind, bins_theta=None, axes=None, **kwargs):
    
    if axes is None:
        fig = plt.figure()
        axes = fig.add_subplot(111)

    a_eff, bins_x_centers, bin_x_errors = compute_effective_area(counts_events, counts_trigger, bins_energy, bins_impact, bins_theta)

    if kind == 'proton':
        spectrum = proton_spectrum(10**bins_x_centers)
        unit = 'Hz'
        
    elif kind == 'gamma':
        
        spectrum = crab_spectrum(10**bins_x_centers)
    
    rate = spectrum * a_eff
    total_rate = np.trapz(rate, 10**bins_x_centers)
    
    if kind == 'gamma':
        
        unit = 'mHz'
        total_rate *= 1E3
        rate *= 1E3
    
    label = kwargs.pop('label', '') + ', Total rate: {:.2f} [{}]'.format(total_rate, unit) 
    axes.plot(10**bins_x_centers, rate, label=label, **kwargs)
    axes.set_xlabel(LABELS['true_energy'])
    axes.set_ylabel('Rate [{}/TeV]'.format(unit))
    axes.set_xscale('log')
    axes.set_yscale('log')
    axes.legend(loc='best')
    
def plot_effective_area(counts_events, counts_trigger, bins_energy, bins_impact, bins_theta=None, axes=None, limit_energy=(-np.inf, np.inf), **kwargs):

    if axes is None:
        fig = plt.figure()
        axes = fig.add_subplot(111)

    a_eff, bins_x_centers, bin_x_errors = compute_effective_area(counts_events, counts_trigger, bins_energy, bins_impact, bins_theta)

    if bins_theta is not None:

        axes.set_ylabel('Effective area [m$^2\cdot$ sr]')

    else:
        axes.set_ylabel('Effective area [m$^2$]')

    mask = np.isfinite(a_eff) * (a_eff > 0) 
    axes.plot(10**bins_x_centers[mask], a_eff[mask], **kwargs)
    axes.set_xlabel('$E$ [TeV]')
    axes.set_xscale('log')
    axes.set_yscale('log')

    return axes

def make_3d_histogram(df, bins_energy, bins_impact, bins_theta, weighted=False):

    energy = np.array(np.log10(df['true_energy']))
    impact = np.array(np.sqrt(df['core_x']**2 + df['core_y']**2))
    theta = np.array(angular_distance(df['alt'], df['az'], df['tel_alt'], df['tel_az']))
    bins = [bins_energy, bins_impact, bins_theta]
    if weighted:
        weights = df['weights']
    else:
        weights = None
    histo, _ = np.histogramdd([energy, impact, theta], bins=bins, weights=weights)
    return histo

def quality_cuts(df, intensity_cut=60, leakage_cut=(0.5, 1.5), wol_cut=0.1):

    n_events = len(df)
    mask = (df['intensity'] >= intensity_cut) & (df['leakage'] <= leakage_cut[1]) & (df['leakage'] >= leakage_cut[0]) # & (df['r'] <= 380)
    mask = mask & (df['wol'] > wol_cut)
    mask = np.arange(len(df))[mask]
    df_out = df.iloc[mask]
    with pd.option_context('mode.use_inf_as_null', True): # Drops infinite values
        df_out = df_out.dropna()

    n_dropped = n_events - len(df_out)
    return df_out

def get_data(file, max_events=None, intensity_cut=60, leakage_cut=(0.5, 1.5), wol_cut=0.1, kind='fit'):

    counts, bins_energy, bins_impact, bins_theta = read_event_histogram(
        file)
    bins = [bins_energy, bins_impact, bins_theta]
    df_triggered, factor = read_hdf5(file, max_events=max_events, kind=kind ,)
    counts = counts * factor
    counts_triggered = make_3d_histogram(df_triggered, bins_energy,
                                               bins_impact, bins_theta, weighted=False)

    df_fitted = get_fitted(df_triggered)
    counts_fitted = make_3d_histogram(df_fitted, bins_energy, bins_impact, bins_theta, weighted=False)

    df_quality = quality_cuts(df_fitted, intensity_cut=intensity_cut, leakage_cut=leakage_cut, wol_cut=wol_cut)

    print("Sample loaded with {} simulated, {} triggered, {} fitted remaining {} after quality cut".format(
            np.sum(counts), len(df_triggered),
            len(df_fitted), len(df_quality)))

    return counts, counts_triggered, counts_fitted, df_fitted, df_quality, bins

In [36]:
directory = '/sst1m/analyzed/mpeshowerfit/zenith_20/cleaning_10_10/'
output_directory = '/home/alispach/figures/sst1m_montecarlo/bis/'
gamma_file = os.path.join(directory, 'gamma.hdf5')
proton_file = os.path.join(directory, 'proton.hdf5')
gamma_diffuse_file = os.path.join(directory, 'gamma_diffuse.hdf5')
format_picture = 'png'

delta_max = 0.13
#directory = '/sst1m/analyzed/mpeshowerfit/lst/dl1/v2/'
#output_directory = '/home/alispach/figures/lst_montecarlo/v2_bis/'
#gamma_file = os.path.join(directory, 'dl1_gamma_converted.h5')
#gamma_diffuse_file = os.path.join(directory, 'dl1_gamma-diffuse_converted.h5')
#proton_file = os.path.join(directory, 'dl1_proton_converted.h5')

In [5]:
!mkdir -p $output_directory

In [6]:
counts_gamma, counts_gamma_triggered, counts_gamma_fitted, df_gamma_fitted, df_gamma_quality, bins_simu = get_data(gamma_file)
counts_gamma_diffuse, counts_gamma_diffuse_triggered, counts_gamma_diffuse_fitted, df_gamma_diffuse_fitted, df_gamma_diffuse_quality, _ = get_data(gamma_diffuse_file)
counts_proton, counts_proton_triggered, counts_proton_fitted, df_proton_fitted, df_proton_quality, _ = get_data(proton_file)

/home/alispach/.conda/envs/digicampipe/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Sample loaded with 97363984.0 simulated, 888131 triggered, 462831 fitted remaining 446726 after quality cut


/home/alispach/.conda/envs/digicampipe/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Sample loaded with 390844013.0 simulated, 1464721 triggered, 883554 fitted remaining 802193 after quality cut


/home/alispach/.conda/envs/digicampipe/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Sample loaded with 443424151.0 simulated, 1134577 triggered, 669509 fitted remaining 603904 after quality cut


In [7]:
rate_gamma = get_rate_gamma(bins_energy=bins_simu[0], bins_impact=bins_simu[1])
rate_proton = get_rate_proton(bins_energy=bins_simu[0], bins_impact=bins_simu[1], bins_theta=bins_simu[2])
rates = [rate_gamma, rate_proton]
weights_gamma = rate_gamma / counts_gamma.sum(axis=-1)
mask = np.isfinite(weights_gamma)
weights_gamma[~mask] = 0
# print(rate_proton.shape)
weights_proton = rate_proton / counts_proton
mask = np.isfinite(weights_proton)
weights_proton[~mask] = 0
weights = [weights_gamma, weights_proton]
dfs = [df_gamma_fitted, df_proton_fitted, df_gamma_diffuse_fitted]
counts = [counts_gamma, counts_proton, counts_gamma_diffuse]
counts_triggered = [counts_gamma_triggered, counts_proton_triggered, counts_gamma_diffuse_triggered]
labels = ['On-axis $\gamma$', 'Diffuse $p$', 'Diffuse $\gamma$']
names = ['gamma', 'proton', 'diffuse_gamma']

/home/alispach/.conda/envs/digicampipe/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
/home/alispach/.conda/envs/digicampipe/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in true_divide
  


# Simulated events

In [37]:
for i, count in enumerate(counts):
    
    c_max = max(count.sum(axis=0).max(), count.sum(axis=1).max(), count.sum(axis=2).max())
    c_max = 10**np.ceil(np.log10(c_max))
    c_min = 1
    axes = plot_3d_histo(count, bins=bins_simu, figsize=figsize, vmin=c_min, vmax=c_max)
    axes.set_xlabel(LABELS['log_energy'])
    axes.set_ylabel(LABELS['impact'])
    axes.set_xlim(bins_simu[0].min(), bins_simu[0].max())
    axes.set_ylim(bins_simu[1].min(), bins_simu[1].max())
    axes.set_title(labels[i])
    #if i > 0:
    #    axes.collections[-1].colorbar.remove()

        
    axes.get_figure().savefig(os.path.join(output_directory, 'simulated_counts_energy_vs_impact_{}.{}'.format(names[i], format_picture)))

    axes = plot_3d_histo(count, bins=bins_simu, figsize=figsize, axis=-2, vmin=c_min, vmax=c_max)
    axes.set_xlabel(LABELS['log_energy'])
    axes.set_ylabel(LABELS['delta'])
    # axes.set_xlim(np.log10(0.5), None)
    
    axes.set_xlim(bins_simu[0].min(), bins_simu[0].max())
    axes.set_ylim(bins_simu[2].min(), bins_simu[2].max())
    axes.set_title(labels[i])
    # axes.collections[-1].colorbar.remove()
    axes.set_ylim(0, delta_max)

    axes.get_figure().savefig(os.path.join(output_directory, 'simulated_counts_energy_vs_delta_{}.{}'.format(names[i], format_picture)))


    axes = plot_3d_histo(count, bins=bins_simu, figsize=figsize, axis=-3, vmin=c_min, vmax=c_max)
    axes.set_xlabel(LABELS['impact'])
    axes.set_ylabel(LABELS['delta'])
    axes.set_title(labels[i])
    axes.set_xlim(bins_simu[1].min(), bins_simu[1].max())
    axes.set_ylim(bins_simu[2].min(), bins_simu[2].max())
    axes.set_ylim(0, delta_max)
    axes.get_figure().savefig(os.path.join(output_directory, 'simulated_counts_impact_vs_delta_{}.{}'.format(names[i], format_picture)))

In [38]:
for i, count in enumerate(weights):

    label = 'Weights [Hz]'
    
    count_0 = count.sum(axis=0)
    count_1 = count.sum(axis=1)
        
    c_max = max(count.sum(axis=0).max(), count.sum(axis=1).max())
    c_min = min(count_0[np.nonzero(count_0)].min(), count_1[np.nonzero(count_1)].min())

    if len(count.shape) > 2:
        count_2 = count.sum(axis=2)
        c_max = max(c_max, count_2.max())
        c_min = min(c_min, count_2[np.nonzero(count_2)].min())
    
    c_max = 10**np.ceil(np.log10(c_max))
    c_min = 10**np.floor(np.log10(c_min))
    
    axes = plot_3d_histo(count, bins=bins_simu, figsize=figsize, label=label, vmin=c_min, vmax=c_max)
    axes.set_xlabel(LABELS['log_energy'])
    axes.set_ylabel(LABELS['impact'])
    # axes.set_xlim(np.log10(0.5), None)
    axes.set_title(labels[i])
    axes.get_figure().savefig(os.path.join(output_directory, 'weigths_energy_vs_impact_{}.{}'.format(names[i], format_picture)))
    
    if i > 0:
    
        axes = plot_3d_histo(count, bins=bins_simu, figsize=figsize, axis=-2,label=label, vmin=c_min, vmax=c_max)
        axes.set_xlabel(LABELS['log_energy'])
        axes.set_ylabel(LABELS['delta'])
        # axes.set_xlim(np.log10(0.5), None)
        axes.set_ylim(0, delta_max)
        axes.set_title(labels[i])
        axes.get_figure().savefig(os.path.join(output_directory, 'weigths_energy_vs_delta_{}.{}'.format(names[i], format_picture)))


        axes = plot_3d_histo(count, bins=bins_simu, figsize=figsize, axis=-3, label=label, vmin=c_min, vmax=c_max)
        axes.set_xlabel(LABELS['impact'])
        axes.set_ylabel(LABELS['delta'])
        axes.set_ylim(0, delta_max)
        axes.set_title(labels[i])
        axes.get_figure().savefig(os.path.join(output_directory, 'weights_impact_vs_delta_{}.{}'.format(names[i], format_picture)))

# Trigger counts

In [39]:
for i, count in enumerate([counts_gamma_triggered, counts_proton_triggered, counts_gamma_diffuse_triggered]):

    count_0 = count.sum(axis=0) / counts[i].sum(axis=0)
    count_1 = count.sum(axis=1) / counts[i].sum(axis=1)
        
    c_max = max(np.nanmax(count_0), np.nanmax(count_0))
    c_min = min(np.nanmin(count_0[np.nonzero(count_0)]), np.nanmin(count_1[np.nonzero(count_1)]))

    if len(count.shape) > 2:
        count_2 = count.sum(axis=2) / counts[i].sum(axis=2)
        c_max = max(c_max, np.nanmax(count_2))
        c_min = min(c_min, np.nanmin(count_2[np.nonzero(count_2)]))
    
    c_max = 10**np.ceil(np.log10(c_max))
    c_min = 10**np.floor(np.log10(c_min))

    c_max = 1
    print(c_min, c_max)
    
    label = 'Trigger probability []'
    axes = plot_3d_histo(count, counts_2=counts[i], bins=bins_simu, figsize=figsize, label=label, vmin=c_min, vmax=c_max)
    axes.set_xlabel(LABELS['log_energy'])
    axes.set_ylabel(LABELS['impact'])
    axes.set_title(labels[i])
    axes.get_figure().savefig(os.path.join(output_directory, 'triggered_probability_energy_vs_impact_{}.{}'.format(names[i], format_picture)))
    
    if i > 0:
    
        axes = plot_3d_histo(count, counts_2=counts[i], bins=bins_simu, figsize=figsize, axis=-2, label=label, vmin=c_min, vmax=c_max)
        axes.set_xlabel(LABELS['log_energy'])
        axes.set_ylabel(LABELS['delta'])
        # axes.set_xlim(np.log10(0.5), None)
        axes.set_title(labels[i])
        axes.set_ylim(0, delta_max)
        axes.get_figure().savefig(os.path.join(output_directory, 'triggered_probability_energy_vs_delta_{}.{}'.format(names[i], format_picture)))


        axes = plot_3d_histo(count, counts_2=counts[i], bins=bins_simu, figsize=figsize, axis=-3, label=label, vmin=c_min, vmax=c_max)
        axes.set_xlabel(LABELS['impact'])
        axes.set_ylabel(LABELS['delta'])
        axes.set_title(labels[i])
        axes.set_ylim(0, delta_max)

        axes.get_figure().savefig(os.path.join(output_directory, 'triggered_probability_impact_vs_delta_{}.{}'.format(names[i], format_picture)))

/home/alispach/.conda/envs/digicampipe/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
/home/alispach/.conda/envs/digicampipe/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.
/home/alispach/.conda/envs/digicampipe/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


1e-07 1
1e-06 1
1e-07 1


# Fit probability

In [40]:
for i, count in enumerate([counts_gamma_fitted, counts_proton_fitted, counts_gamma_diffuse_fitted]):

    count_0 = count.sum(axis=0) / counts[i].sum(axis=0)
    count_1 = count.sum(axis=1) / counts[i].sum(axis=1)
        
    c_max = max(np.nanmax(count_0), np.nanmax(count_0))
    c_min = min(np.nanmin(count_0[np.nonzero(count_0)]), np.nanmin(count_1[np.nonzero(count_1)]))

    if len(count.shape) > 2:
        count_2 = count.sum(axis=2) / counts[i].sum(axis=2)
        c_max = max(c_max, np.nanmax(count_2))
        c_min = min(c_min, np.nanmin(count_2[np.nonzero(count_2)]))
    
    c_max = 10**np.ceil(np.log10(c_max))
    c_min = 10**np.floor(np.log10(c_min))

    c_max = 1
    print(c_min, c_max)
    
    label = 'Fit probability []'
    axes = plot_3d_histo(count, counts_2=counts[i], bins=bins_simu, figsize=figsize, label=label, vmin=c_min, vmax=c_max)
    axes.set_xlabel(LABELS['log_energy'])
    axes.set_ylabel(LABELS['impact'])
    # axes.set_xlim(np.log10(0.5), None)
    axes.set_title(labels[i])
    axes.get_figure().savefig(os.path.join(output_directory, 'fitted_probability_energy_vs_impact_{}.{}'.format(names[i], format_picture)))
    
    
    if i > 0:
        axes = plot_3d_histo(count, counts_2=counts[i], bins=bins_simu, figsize=figsize, axis=-2, label=label, vmin=c_min, vmax=c_max)
        axes.set_xlabel(LABELS['log_energy'])
        axes.set_ylabel(LABELS['delta'])
        # axes.set_xlim(np.log10(0.5), None)
        axes.set_ylim(0, delta_max)
        axes.set_title(labels[i])
        axes.get_figure().savefig(os.path.join(output_directory, 'fitted_probability_energy_vs_delta_{}.{}'.format(names[i], format_picture)))


        axes = plot_3d_histo(count, counts_2=counts[i], bins=bins_simu, figsize=figsize, axis=-3, label=label, vmin=c_min, vmax=c_max)
        axes.set_xlabel(LABELS['impact'])
        axes.set_ylabel(LABELS['delta'])
        axes.set_ylim(0, delta_max)
        axes.set_title(labels[i])
        axes.get_figure().savefig(os.path.join(output_directory, 'fitted_probability_impact_vs_delta_{}.{}'.format(names[i], format_picture)))

/home/alispach/.conda/envs/digicampipe/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
/home/alispach/.conda/envs/digicampipe/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.
/home/alispach/.conda/envs/digicampipe/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.
/home/alispach/Documents/PhD/ctasoft/digicampipe/digicampipe/visualization/machine_learning.py:629: RuntimeWarning: invalid value encountered in true_divide
  data = data_1 / data_2


1e-07 1
1e-07 1
1e-07 1


# Quality cut probability

In [41]:
options = {'histtype': 'step', 'lw':3, 'density': True}
plt.figure(figsize=figsize)
bins = np.logspace(0, 6, num=100)
size_cut = 60
wol_cut = 0.1
leakage_min, leakage_max = 0.5, 1.5
for i, df in enumerate(dfs):
    plt.hist(df['intensity'], bins=bins, label=labels[i], **options)
plt.axvline(size_cut, label='Lower cut', linestyle='--', color='k')
plt.yscale('log')
plt.xscale('log')
plt.xlabel('Size [p.e.]')
plt.ylabel('Normalized counts []')
plt.legend(loc='best')
plt.savefig(os.path.join(output_directory, 'size_distribution_triggered.{}'.format(format_picture)))

plt.figure(figsize=figsize)
bins = np.linspace(0, 1, num=100)
for i, df in enumerate(dfs):
    plt.hist(df['wol'], bins=bins, label=labels[i], **options)
plt.axvline(wol_cut, label='Lower cut', linestyle='--', color='k')

plt.yscale('log')
plt.xlabel('Width over length []')
plt.ylabel('Normalized counts []')
plt.legend(loc='best')
plt.savefig(os.path.join(output_directory, 'wol_distribution_triggered.{}'.format(format_picture)))


plt.figure(figsize=figsize)
bins = np.linspace(0, 5, num=100)
for i, df in enumerate(dfs):
    plt.hist(df['leakage'], bins=bins, label=labels[i], **options)
plt.axvline(leakage_min, label='Lower cut', linestyle='--', color='k')
plt.axvline(leakage_max, label='Upper cut', linestyle='-.', color='k')

plt.yscale('log')
plt.xlabel('Containment []')
plt.ylabel('Normalized counts []')
plt.legend(loc='best')
plt.savefig(os.path.join(output_directory, 'containement_distribution_triggered.{}'.format(format_picture)))

In [16]:

for i, label in enumerate(labels):

    bins_x = np.linspace(0, 5, num=100)
    plt.figure(figsize=figsize)
    bins_y = np.linspace(0, 1, num=100)
    plt.title(label)
    plt.hist2d(dfs[i]['leakage'], dfs[i]['wol'], bins=[bins_x, bins_y], norm=LogNorm())
    plt.xlabel('Containment []')
    plt.ylabel('Width over length []')
    plt.colorbar(label='Counts []')
    plt.legend(loc='best')
    plt.savefig(os.path.join(output_directory, 'containement_vs_wol_distribution_triggered_{}.{}'.format(names[i], format_picture)))
    
    plt.figure(figsize=figsize)
    bins_y = np.linspace(0, 1000, num=100)
    plt.title(label)
    plt.hist2d(dfs[i]['leakage'], dfs[i]['r'], bins=[bins_x, bins_y], norm=LogNorm())
    plt.xlabel('Containment []')
    plt.ylabel('$r_{CM}$ [mm]')
    plt.colorbar(label='Counts []')
    plt.legend(loc='best')
    plt.savefig(os.path.join(output_directory, 'containement_vs_r_distribution_triggered_{}.{}'.format(names[i], format_picture)))
    
    
    plt.figure(figsize=figsize)
    plt.title(label)
    bins = np.logspace(0, 6, num=100)
    bins_y = np.linspace(0, 1, num=100)
    plt.hist2d(dfs[i]['intensity'], dfs[i]['wol'], bins=[bins, bins_y], norm=LogNorm())
    plt.xlabel('Size [p.e.]')
    plt.xscale('log')
    plt.ylabel('width over length []')
    plt.colorbar(label='Counts []')
    plt.legend(loc='best')
    plt.savefig(os.path.join(output_directory, 'intensity_vs_wol_distribution_triggered_{}.{}'.format(names[i], format_picture)))
    
    plt.figure(figsize=figsize)
    bins = np.logspace(0, 6, num=100)
    bins_y = np.linspace(0, 5, num=100)

    plt.title(label)
    plt.hist2d(dfs[i]['intensity'], dfs[i]['leakage'], bins=[bins, bins_y], norm=LogNorm())
    plt.ylabel('Containment []')
    plt.xlabel('Size [p.e.]')
    plt.xscale('log')
    plt.colorbar(label='Counts []')
    plt.legend(loc='best')
    plt.savefig(os.path.join(output_directory, 'intensity_vs_containement_distribution_triggered_{}.pdf'.format(names[i])))
 

/home/alispach/.conda/envs/digicampipe/lib/python3.6/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [42]:
counts_gamma_quality_cut, _ = np.histogramdd([np.log10(df_gamma_quality['true_energy']), df_gamma_quality['impact'], df_gamma_quality['delta']], bins=bins_simu)
counts_proton_quality_cut, _ = np.histogramdd([np.log10(df_proton_quality['true_energy']), df_proton_quality['impact'], df_proton_quality['delta']], bins=bins_simu)
counts_gamma_diffuse_quality_cut, _ = np.histogramdd([np.log10(df_gamma_diffuse_quality['true_energy']), df_gamma_diffuse_quality['impact'], df_gamma_diffuse_quality['delta']], bins=bins_simu)
counts_fitted = [counts_gamma_fitted, counts_proton_fitted, counts_gamma_diffuse_fitted]
counts_quality = [counts_gamma_quality_cut, counts_proton_quality_cut, counts_gamma_diffuse_quality_cut]

In [43]:
for i, count in enumerate([counts_gamma_quality_cut, counts_proton_quality_cut, counts_gamma_diffuse_quality_cut]):

    count_0 = count.sum(axis=0) / counts[i].sum(axis=0)
    count_1 = count.sum(axis=1) / counts[i].sum(axis=1)
        
    c_max = max(np.nanmax(count_0), np.nanmax(count_0))
    c_min = min(np.nanmin(count_0[np.nonzero(count_0)]), np.nanmin(count_1[np.nonzero(count_1)]))

    if len(count.shape) > 2:
        count_2 = count.sum(axis=2) / counts[i].sum(axis=2)
        c_max = max(c_max, np.nanmax(count_2))
        c_min = min(c_min, np.nanmin(count_2[np.nonzero(count_2)]))
    
    c_max = 10**np.ceil(np.log10(c_max))
    c_min = 10**np.floor(np.log10(c_min))

    c_max = 1
    print(c_min, c_max)
    
    label = 'Survival probability []'
    axes = plot_3d_histo(count, counts_2=counts[i], bins=bins_simu, figsize=figsize, label=label, vmin=c_min, vmax=c_max)
    axes.set_xlabel(LABELS['log_energy'])
    axes.set_ylabel(LABELS['impact'])
    # axes.set_xlim(np.log10(0.5), None)
    axes.set_title(labels[i])
    axes.get_figure().savefig(os.path.join(output_directory, 'quality_cut_probability_energy_vs_impact_{}.{}'.format(names[i], format_picture)))
    
    
    if i > 0:
        axes = plot_3d_histo(count, counts_2=counts[i], bins=bins_simu, figsize=figsize, axis=-2, label=label, vmin=c_min, vmax=c_max)
        axes.set_xlabel(LABELS['log_energy'])
        axes.set_ylabel(LABELS['delta'])
        # axes.set_xlim(np.log10(0.5), None)
        axes.set_ylim(0, delta_max)
        axes.set_title(labels[i])
        axes.get_figure().savefig(os.path.join(output_directory, 'quality_cut_probability_energy_vs_delta_{}.{}'.format(names[i], format_picture)))


        axes = plot_3d_histo(count, counts_2=counts[i], bins=bins_simu, figsize=figsize, axis=-3, label=label, vmin=c_min, vmax=c_max)
        axes.set_xlabel(LABELS['impact'])
        axes.set_ylabel(LABELS['delta'])
        axes.set_ylim(0, delta_max)
        axes.set_title(labels[i])
        axes.get_figure().savefig(os.path.join(output_directory, 'quality_cut_probability_impact_vs_delta_{}.{}'.format(names[i], format_picture)))

/home/alispach/.conda/envs/digicampipe/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
/home/alispach/.conda/envs/digicampipe/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.
/home/alispach/.conda/envs/digicampipe/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.
/home/alispach/Documents/PhD/ctasoft/digicampipe/digicampipe/visualization/machine_learning.py:629: RuntimeWarning: invalid value encountered in true_divide
  data = data_1 / data_2


1e-07 1
1e-07 1
1e-07 1


# Effective areas

In [56]:
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

for i, count in enumerate(counts):

    fig = plt.figure(figsize=figsize)
    axes = fig.add_subplot(111)
    axes.set_title(labels[i])
    options = {'color': colors[i], 'marker': None}
    bins_theta = bins_simu[2] if i > 0 else None
    plot_effective_area(count, counts_triggered[i],
                                   bins_energy=bins_simu[0],
                                   bins_impact=bins_simu[1],
                                    bins_theta=bins_theta,
                                   label='Triggered', axes=axes,
                                    linestyle='-.',
                                    **options)
    
    plot_effective_area(count, counts_fitted[i],
                                   bins_energy=bins_simu[0],
                                   bins_impact=bins_simu[1],
                                                            bins_theta=bins_theta,

                                   label='Fitted', axes=axes,
                                    linestyle='--',
                       **options)


    plot_effective_area(count, counts_quality[i],
                                   bins_energy=bins_simu[0],
                                   bins_impact=bins_simu[1],
                                                            bins_theta=bins_theta,

                                   label='Cleaned data', axes=axes , linestyle='-', **options)

    axes.set_xlabel('$E$ [TeV]')
    axes.legend(loc='best')
    axes.set_ylim(1, 1E7)
    axes.set_xlim(10**bins_simu[0].min(), 10**bins_simu[0].max())
    fig.savefig(os.path.join(output_directory, 'effective_area_{}.{}'.format(names[i], format_picture)))
    
    if i < 2:

        fig = plt.figure(figsize=figsize)
        axes = fig.add_subplot(111)
        axes.set_title(labels[i])
        options = {'color': colors[i], 'marker': None}
        bins_theta = bins_simu[2] if i > 0 else None
    
    
        plot_trigger_rate(count, counts_triggered[i],
                                       bins_energy=bins_simu[0],
                                       bins_impact=bins_simu[1],
                                        bins_theta=bins_theta,
                                       label='Triggered', axes=axes,
                                        linestyle='-.', kind=names[i],
                                        **options)

        plot_trigger_rate(count, counts_fitted[i],
                                       bins_energy=bins_simu[0],
                                       bins_impact=bins_simu[1],
                                                                bins_theta=bins_theta,

                                       label='Fitted', axes=axes,
                                        linestyle='--', kind=names[i],
                           **options)


        plot_trigger_rate(count, counts_quality[i],
                                       bins_energy=bins_simu[0],
                                       bins_impact=bins_simu[1],
                                                                bins_theta=bins_theta,

                                       label='Cleaned data', axes=axes , linestyle='-', kind=names[i], **options)

        axes.set_ylim(1E-7, 1E1)
        axes.set_xlim(10**bins_simu[0].min(), 10**bins_simu[0].max())
        fig.savefig(os.path.join(output_directory, 'trigger_rate_vs_energy_{}.{}'.format(names[i], format_picture)))

/home/alispach/Documents/PhD/ctasoft/digicampipe/digicampipe/visualization/machine_learning.py:667: RuntimeWarning: invalid value encountered in true_divide
  prob = counts_trigger / counts_events
/home/alispach/Documents/PhD/ctasoft/digicampipe/digicampipe/visualization/machine_learning.py:667: RuntimeWarning: divide by zero encountered in true_divide
  prob = counts_trigger / counts_events


# Image parameters distribution after quality cuts

In [51]:
image_parameters = ['y', 'x', 't_68', 'intensity', 'wol', 'r', 'phi', 'psi', 'width', 'length', 'alpha', 'area']
for param in image_parameters:
    
    fig = plt.figure(figsize=figsize)
    axes = fig.add_subplot(111)
    axes.set_yscale('log')
    axes.yaxis.set_major_formatter(matplotlib.ticker.LogFormatterMathtext(labelOnlyBase=True,))
    
    bins = 100
    h_max = 0
    h_min = np.inf
    for i, df in enumerate([df_gamma_quality, df_proton_quality, df_gamma_diffuse_quality]):
        
        x = df[param]
        label_x = LABELS[param]

        if param == 't_68':
            bins = np.linspace(0, 200, num=400)
        if param == 'phi':
            x = np.arctan2(df['y'], df['x'])
        if param == 'intensity':
            x = np.log10(x)
            label_x = r'$\log_{10} \left( \frac{\mu}{p.e.} \right)$'
                
        H = axes.hist(x, bins=bins, density=True, label=labels[i], lw=2, histtype='step')
        bins = H[1]
        h_max = max(H[0].max(), h_max)
        h_min = min(H[0].min(), h_min)
    axes.set_xlabel(label_x)
    axes.set_ylabel('Normalized counts []')
    axes.legend(loc='best')

    h_min = 10**np.floor(np.log10(h_min))
    h_max = 10**np.ceil(np.log10(h_max))
    print(h_min, h_max)
    axes.set_ylim(h_min, h_max)
    fig.savefig(os.path.join(output_directory,'histogram_{}_all_particle_after_quality_cuts.{}'.format(param, format_picture)))

1e-07 0.01
1e-06 0.01


/home/alispach/.conda/envs/digicampipe/lib/python3.6/site-packages/ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in log10
/home/alispach/.conda/envs/digicampipe/lib/python3.6/site-packages/ipykernel_launcher.py:36: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.


0.0 1.0
0.0 10.0
0.01 10.0
1e-06 0.1
0.1 1.0
0.1 1.0
0.0 1.0
0.0 0.1
0.001 100.0
0.0 0.001


In [ ]:
import importlib
importlib.reload(matplotlib)
matplotlib.rcParams
%matplotlib inline
# matplotlib.matplotlib_fname()

In [57]:
plt.figure(figsize=figsize)
bins_alpha = np.linspace(0, np.pi/2, num=100)
plt.title(labels[0])
plt.hist2d(df_gamma_quality['alpha'], df_gamma_quality['wol'], bins=(bins_alpha, np.linspace(0, 1, num=100)), norm=LogNorm())
plt.xlabel(LABELS['alpha'])
plt.ylabel(LABELS['wol'])
plt.colorbar(label='count []')
plt.savefig(os.path.join(output_directory, 'alpha_vs_wol_gamma_quality_cuts.{}'.format(format_picture)))
plt.figure(figsize=figsize)
plt.title(labels[0])
H=plt.hist2d(df_gamma_quality['alpha'], df_gamma_quality['t_68'], bins=[bins_alpha, np.linspace(0, 30, num=100)], norm=LogNorm())
plt.xlabel(LABELS['alpha'])
plt.ylabel(LABELS['t_68'])
plt.colorbar(label='count []')
plt.savefig(os.path.join(output_directory, 'alpha_vs_t_68_gamma_quality_cuts.{}'.format(format_picture)))

In [ ]:
df_gamma_diffuse_fitted.columns
